In [ ]:
# Import necessary libraries and packages
import math
import matplotlib.pyplot as plt
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

from qiskit import Aer, IBMQ, QuantumCircuit
from qiskit.primitives import Estimator
from qiskit.providers.aer import StatevectorSimulator
from qiskit.utils import QuantumInstance
import qiskit.providers as providers
#from qiskit import provider
from qiskit.tools.jupyter import *
from qiskit.visualization import *

# Import Qiskit libraries for VQE
from qiskit.algorithms import MinimumEigensolverResult, VQE
from qiskit.algorithms.optimizers import SLSQP, SPSA

# Import Qiskit Nature libraries
from qiskit_nature.algorithms import GroundStateEigensolver, VQEUCCFactory
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.circuit.library import UCC, UCCSD, HartreeFock
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import BravyiKitaevMapper, JordanWignerMapper, ParityMapper
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.transformers.second_quantization.electronic import ActiveSpaceTransformer,FreezeCoreTransformer
from qiskit.utils import algorithm_globals
from qiskit.algorithms.eigensolvers import VQD
from qiskit.algorithms.state_fidelities import ComputeUncompute
from qiskit.primitives import Sampler, Estimator
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B
from qiskit import IBMQ

In [ ]:
provider = IBMQ.load_account()
#backend = provider.get_backend('ibmq_qasm_simulator')
backend = Aer.get_backend('aer_simulator')
#qi = QuantumInstance(backend=backend,shots = 10000, seed_simulator=10)

In [ ]:
def dipole_moment(cyclic,charge,mult):

  algorithm_globals.random_seed = 1024
  algorithm_globals.massive=True 

  molecule = Molecule(geometry=cyclic,
                              charge=charge, 
                              multiplicity=mult)
  ### Defining the driver
  driver = ElectronicStructureMoleculeDriver(molecule, basis="sto3g", driver_type=ElectronicStructureDriverType.PYSCF) # This is an example of using STO-3G basis set, and performing preliminary Quantum Chemistry calculation using PySCF
  
  problem_reduced = ElectronicStructureProblem(driver, [FreezeCoreTransformer(freeze_core=True)])
  second_q_ops_reduced = problem_reduced.second_q_ops()

  # Set the mapper to qubits
  mapper = JordanWignerMapper() # This is the example of parity mapping

  # Set the qubit converter with two qubit reduction to reduce the computational cost 
  converter = QubitConverter(mapper,z2symmetry_reduction='auto')    

  # Compute the Hamitonian in qubit form
  qubit_op_parity = converter.convert(second_q_ops_reduced.get('ElectronicEnergy'), sector_locator=problem_reduced.symmetry_sector_locator)

  init_state = HartreeFock(num_spin_orbitals=problem_reduced.num_spin_orbitals, num_particles=problem_reduced.num_particles , qubit_converter= converter)

  vqe_factory = VQEUCCFactory(quantum_instance=backend,optimizer=SPSA(maxiter=50),ansatz=UCC(qubit_converter=converter, num_particles=problem_reduced.num_particles,num_spin_orbitals=problem_reduced.num_spin_orbitals,excitations='sd', initial_state=init_state)) # This is an example of UCC"SD" ansatz

  solver = GroundStateEigensolver(converter, vqe_factory)
   
  result = solver.solve(problem_reduced)    

  dip_tot = result.total_dipole_moment_in_debye[0]
  min_energy = result.total_energies[0]

  return dip_tot,min_energy

In [ ]:
dip = []
Energy =[]
dist =  np.linspace(1,4,7)
Charge = 0
Multiplicity = 1

for i in dist:
    cyclic = [["Li",[0.000,1.9748,-0.5280]],
          ["S",[0.0000,0.000,0.1980]],
          ["Li",[0.0000,-i,-0.5280]]]
    dipole,energy = dipole_moment(cyclic,Charge,Multiplicity)
    Energy.append(energy)
    dip.append(dipole)
    print('Dipole (debye) for '+str(i)+' : ', dipole)
    print('Energy : ', energy)

In [ ]:
dist = np.linspace(1,4,7)
bond_length=[np.sqrt(i**2+(0.1980+0.5280)**2) for i in dist]

In [ ]:
plt.plot(bond_length, Energy, '-o')
plt.savefig('energy_JWT_Z2_Li2S.png')
